# Data Preprocessing for SUEWS Tutorial

This notebook demonstrates how to preprocess meteorological data for SUEWS simulations using the MCP server.

## What you'll learn:
- How to assess raw meteorological data quality
- How to convert between different data formats
- How to validate energy balance data
- How to handle missing data and gaps
- How to prepare SUEWS-ready forcing files

## Prerequisites:
- SUEWS MCP server running
- Sample meteorological data files (or synthetic data)
- Python packages: `mcp`, `pandas`, `matplotlib`, `numpy`

In [ ]:
# Import required packages
import asyncio
from mcp import create_client
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
import os

# For Jupyter notebooks
import nest_asyncio
nest_asyncio.apply()

print("📦 Packages imported successfully")
print("🔧 Ready for data preprocessing workflow")

## Step 1: Create Sample Data Files

Since you might not have real meteorological data files, let's create some realistic sample data with various common issues.

In [ ]:
def create_sample_weather_data():
    """Create realistic sample meteorological data with some common issues."""
    
    # Create directory for sample data
    os.makedirs('sample_data', exist_ok=True)
    
    print("🏗️ Creating sample meteorological data files...")
    
    # Generate hourly data for one month (July 2023)
    start_date = datetime(2023, 7, 1)
    end_date = datetime(2023, 7, 31, 23)
    dates = pd.date_range(start_date, end_date, freq='H')
    n_points = len(dates)
    
    # Generate realistic meteorological variables with daily patterns
    hour_of_day = dates.hour
    day_of_year = dates.dayofyear
    
    # Base patterns
    temp_base = 20 + 8 * np.sin(2 * np.pi * (hour_of_day - 6) / 24)  # Diurnal temperature
    temp_noise = np.random.normal(0, 2, n_points)  # Weather variability
    temperature = temp_base + temp_noise
    
    # Relative humidity (inversely related to temperature)
    rh_base = 70 - 20 * np.sin(2 * np.pi * (hour_of_day - 6) / 24)
    humidity = np.clip(rh_base + np.random.normal(0, 10, n_points), 20, 95)
    
    # Wind speed (higher during day)
    wind_base = 3 + 2 * np.sin(2 * np.pi * (hour_of_day - 10) / 24)
    wind_speed = np.maximum(0.5, wind_base + np.random.normal(0, 1, n_points))
    
    # Wind direction (some variability)
    wind_direction = 220 + 30 * np.sin(2 * np.pi * hour_of_day / 24) + np.random.normal(0, 20, n_points)
    wind_direction = wind_direction % 360
    
    # Atmospheric pressure
    pressure = 101.3 + np.random.normal(0, 2, n_points)
    
    # Solar radiation (zero at night, peak at noon)
    solar_angle = np.maximum(0, np.sin(np.pi * (hour_of_day - 6) / 12))
    solar_radiation = 800 * solar_angle * (0.7 + 0.3 * np.random.random(n_points))
    
    # Precipitation (random events)
    rain_prob = np.random.random(n_points)
    precipitation = np.where(rain_prob < 0.05, np.random.exponential(2, n_points), 0)
    
    # Energy fluxes (for flux tower data)
    net_radiation = 0.7 * solar_radiation - 50  # Net radiation
    sensible_heat = 0.3 * np.maximum(0, net_radiation) + np.random.normal(0, 15, n_points)
    latent_heat = 0.4 * np.maximum(0, net_radiation) + np.random.normal(0, 20, n_points)
    
    # Create weather station CSV (with some issues)
    weather_data = pd.DataFrame({
        'timestamp': dates,
        'air_temperature': temperature,
        'relative_humidity': humidity,
        'wind_speed': wind_speed,
        'wind_direction': wind_direction,
        'pressure': pressure,
        'global_radiation': solar_radiation,
        'precipitation': precipitation
    })
    
    # Introduce some data quality issues
    # 1. Missing data
    missing_indices = np.random.choice(weather_data.index, size=20, replace=False)
    weather_data.loc[missing_indices, 'relative_humidity'] = np.nan
    
    # 2. Some outliers
    outlier_indices = np.random.choice(weather_data.index, size=5, replace=False)
    weather_data.loc[outlier_indices, 'air_temperature'] = 45  # Unrealistic temperature
    
    # 3. Some negative radiation values (instrument error)
    night_indices = weather_data[weather_data['global_radiation'] < 10].index
    error_indices = np.random.choice(night_indices, size=10, replace=False)
    weather_data.loc[error_indices, 'global_radiation'] = -20
    
    # Save weather station data
    weather_data.to_csv('sample_data/weather_station.csv', index=False)
    print(f"✅ Created weather_station.csv ({len(weather_data)} records)")
    
    # Create flux tower data (Excel format)
    flux_data = pd.DataFrame({
        'TIMESTAMP': dates,
        'TA_1_1_1': temperature,  # Air temperature
        'RH_1_1_1': humidity,    # Relative humidity
        'WS_1_1_1': wind_speed,  # Wind speed
        'WD_1_1_1': wind_direction,  # Wind direction
        'PA_1_1_1': pressure,    # Pressure
        'SW_IN_1_1_1': solar_radiation,  # Shortwave in
        'NETRAD_1_1_1': net_radiation,   # Net radiation
        'H_1_1_1': sensible_heat,        # Sensible heat
        'LE_1_1_1': latent_heat,         # Latent heat
        'P_1_1_1': precipitation         # Precipitation
    })
    
    # Introduce energy balance issues
    # Some periods with poor closure
    bad_closure_mask = np.random.random(n_points) < 0.1  # 10% of data
    flux_data.loc[bad_closure_mask, 'H_1_1_1'] *= 1.5  # Overestimate sensible heat
    
    flux_data.to_excel('sample_data/flux_tower.xlsx', index=False)
    print(f"✅ Created flux_tower.xlsx ({len(flux_data)} records)")
    
    # Create text file with irregular time steps
    irregular_dates = []
    current_date = start_date
    while current_date <= end_date:
        irregular_dates.append(current_date)
        # Random time step between 30min and 2 hours
        delta_minutes = np.random.randint(30, 120)
        current_date += timedelta(minutes=delta_minutes)
    
    irregular_data = pd.DataFrame({
        'datetime': irregular_dates[:500],  # Limit to 500 points
        'Temperature': np.random.normal(20, 5, 500),
        'Humidity': np.random.normal(60, 15, 500),
        'WindSpeed': np.random.normal(3, 1, 500)
    })
    
    irregular_data.to_csv('sample_data/irregular_timesteps.txt', sep='\t', index=False)
    print(f"✅ Created irregular_timesteps.txt ({len(irregular_data)} records)")
    
    print("\n📁 Sample data files created in 'sample_data/' directory")
    print("\n🔍 Data quality issues intentionally included:")
    print("   • Missing humidity values (weather station)")
    print("   • Temperature outliers (weather station)")
    print("   • Negative radiation values (weather station)")
    print("   • Poor energy balance closure (flux tower)")
    print("   • Irregular time steps (text file)")

# Create sample data
create_sample_weather_data()

## Step 2: Connect to MCP Server

Let's connect to the SUEWS MCP server and check that data preprocessing tools are available.

In [ ]:
async def check_preprocessing_tools():
    """Check that data preprocessing tools are available."""
    try:
        async with create_client("suews-mcp") as client:
            print("✅ Connected to SUEWS MCP server")
            
            # List tools and check for preprocessing capabilities
            tools = await client.list_tools()
            
            preprocessing_tools = [
                'preprocess_forcing',
                'convert_data_format',
                'validate_config'
            ]
            
            available_tools = [tool.name for tool in tools.tools]
            
            print("\n🔧 Data preprocessing tools:")
            for tool_name in preprocessing_tools:
                if tool_name in available_tools:
                    print(f"   ✅ {tool_name}")
                else:
                    print(f"   ❌ {tool_name} (not available)")
            
            return True
            
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return False

# Check connection and tools
connection_ok = await check_preprocessing_tools()

## Step 3: Assess Raw Data Quality

Let's use the preprocessing tool to assess the quality of our sample weather station data.

In [ ]:
async def assess_weather_station_data():
    """Assess quality of weather station data."""
    if not connection_ok:
        print("❌ Skipping - no MCP connection")
        return
    
    async with create_client("suews-mcp") as client:
        print("🔍 Assessing weather station data quality...")
        
        try:
            # First, let's try to preprocess without fixing issues
            assessment = await client.call_tool("preprocess_forcing", {
                "input_file": "sample_data/weather_station.csv",
                "output_file": "sample_data/weather_station_assessed.txt",
                "validate_energy_balance": False,  # No energy balance data
                "auto_fix_issues": False,  # Just assess first
                "target_timestep": 3600  # Hourly
            })
            
            print("📊 Data Quality Assessment:")
            print("=" * 50)
            print(assessment.content[0].text)
            
            return assessment
            
        except Exception as e:
            print(f"⚠️ Assessment failed: {e}")
            print("This might be due to file format issues or missing MCP tools")
            return None

# Assess data quality
if connection_ok:
    assessment_result = await assess_weather_station_data()

## Step 4: Convert Data Formats

Let's convert the Excel flux tower data to SUEWS format, demonstrating column mapping.

In [ ]:
async def convert_flux_tower_data():
    """Convert flux tower Excel data to SUEWS format."""
    if not connection_ok:
        print("❌ Skipping - no MCP connection")
        return
    
    async with create_client("suews-mcp") as client:
        print("🔄 Converting flux tower data format...")
        
        try:
            conversion = await client.call_tool("convert_data_format", {
                "input_file": "sample_data/flux_tower.xlsx",
                "output_file": "sample_data/flux_tower_suews.txt",
                "input_format": "excel",
                "output_format": "suews_txt",
                "column_mapping": {
                    "TIMESTAMP": "datetime",
                    "TA_1_1_1": "Tair",         # Air temperature
                    "RH_1_1_1": "RH",           # Relative humidity
                    "WS_1_1_1": "U",            # Wind speed
                    "WD_1_1_1": "WDir",         # Wind direction
                    "PA_1_1_1": "Pres",         # Pressure
                    "SW_IN_1_1_1": "Kdown",     # Shortwave incoming
                    "NETRAD_1_1_1": "QN",       # Net radiation
                    "H_1_1_1": "QH",            # Sensible heat flux
                    "LE_1_1_1": "QE",           # Latent heat flux
                    "P_1_1_1": "Rain"           # Precipitation
                }
            })
            
            print("📄 Format Conversion Results:")
            print("=" * 40)
            print(conversion.content[0].text)
            
            return conversion
            
        except Exception as e:
            print(f"⚠️ Conversion failed: {e}")
            print("This might be due to file format issues or missing dependencies")
            return None

# Convert flux tower data
if connection_ok:
    conversion_result = await convert_flux_tower_data()

## Step 5: Energy Balance Validation

Now let's validate the energy balance in the converted flux tower data.

In [ ]:
async def validate_energy_balance():
    """Validate energy balance in flux tower data."""
    if not connection_ok:
        print("❌ Skipping - no MCP connection")
        return
    
    async with create_client("suews-mcp") as client:
        print("⚖️ Validating energy balance...")
        
        try:
            validation = await client.call_tool("preprocess_forcing", {
                "input_file": "sample_data/flux_tower_suews.txt",
                "output_file": "sample_data/flux_tower_validated.txt",
                "validate_energy_balance": True,  # Check QN = QH + QE + QS
                "auto_fix_issues": False,  # Just validate first
                "target_timestep": 3600
            })
            
            print("⚖️ Energy Balance Validation Results:")
            print("=" * 45)
            print(validation.content[0].text)
            
            return validation
            
        except Exception as e:
            print(f"⚠️ Energy balance validation failed: {e}")
            print("This might be because the converted file doesn't exist or has format issues")
            return None

# Validate energy balance
if connection_ok:
    energy_validation = await validate_energy_balance()

## Step 6: Fix Data Issues

Let's now apply automatic fixes to the weather station data.

In [ ]:
async def fix_weather_station_issues():
    """Apply automatic fixes to weather station data."""
    if not connection_ok:
        print("❌ Skipping - no MCP connection")
        return
    
    async with create_client("suews-mcp") as client:
        print("🔧 Applying automatic fixes to weather station data...")
        
        try:
            fixing = await client.call_tool("preprocess_forcing", {
                "input_file": "sample_data/weather_station.csv",
                "output_file": "sample_data/weather_station_fixed.txt",
                "validate_energy_balance": False,
                "auto_fix_issues": True,  # Now apply fixes
                "target_timestep": 3600
            })
            
            print("🛠️ Data Fixing Results:")
            print("=" * 30)
            print(fixing.content[0].text)
            
            return fixing
            
        except Exception as e:
            print(f"⚠️ Data fixing failed: {e}")
            return None

# Fix data issues
if connection_ok:
    fixing_result = await fix_weather_station_issues()

## Step 7: Compare Before and After

Let's load the original and processed data to see what changes were made.

In [ ]:
def compare_before_after():
    """Compare original and processed data to show improvements."""
    
    try:
        # Load original data
        original_data = pd.read_csv('sample_data/weather_station.csv', parse_dates=['timestamp'])
        print("✅ Loaded original weather station data")
        
        # Try to load processed data
        processed_files = [
            'sample_data/weather_station_fixed.txt',
            'sample_data/weather_station_assessed.txt'
        ]
        
        processed_data = None
        for file_path in processed_files:
            try:
                processed_data = pd.read_csv(file_path, delim_whitespace=True)
                print(f"✅ Loaded processed data from {file_path}")
                break
            except FileNotFoundError:
                continue
        
        if processed_data is None:
            print("⚠️ No processed data file found, creating comparison with original data only")
            processed_data = original_data.copy()
        
        # Create comparison plots
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle('Data Preprocessing Comparison: Before vs After', fontsize=16, fontweight='bold')
        
        # Plot 1: Temperature comparison
        ax1 = axes[0, 0]
        ax1.plot(original_data['timestamp'], original_data['air_temperature'], 
                'b-', alpha=0.7, label='Original', linewidth=1)
        
        if 'Tair' in processed_data.columns:
            ax1.plot(original_data['timestamp'], processed_data['Tair'], 
                    'r-', alpha=0.8, label='Processed', linewidth=1.5)
        
        ax1.set_title('Air Temperature Comparison', fontweight='bold')
        ax1.set_ylabel('Temperature (°C)')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Highlight outliers in original data
        outliers = original_data['air_temperature'] > 35
        if outliers.any():
            ax1.scatter(original_data.loc[outliers, 'timestamp'], 
                       original_data.loc[outliers, 'air_temperature'],
                       color='red', s=50, marker='x', label='Outliers', zorder=5)
        
        # Plot 2: Humidity with missing data
        ax2 = axes[0, 1]
        ax2.plot(original_data['timestamp'], original_data['relative_humidity'], 
                'b-', alpha=0.7, label='Original (with gaps)', linewidth=1)
        
        if 'RH' in processed_data.columns:
            ax2.plot(original_data['timestamp'], processed_data['RH'], 
                    'g-', alpha=0.8, label='Gap-filled', linewidth=1.5)
        
        ax2.set_title('Relative Humidity (Gap Filling)', fontweight='bold')
        ax2.set_ylabel('Relative Humidity (%)')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
        
        # Plot 3: Solar radiation (fixing negative values)
        ax3 = axes[1, 0]
        ax3.plot(original_data['timestamp'], original_data['global_radiation'], 
                'b-', alpha=0.7, label='Original', linewidth=1)
        
        if 'Kdown' in processed_data.columns:
            ax3.plot(original_data['timestamp'], processed_data['Kdown'], 
                    'orange', alpha=0.8, label='Corrected', linewidth=1.5)
        
        ax3.set_title('Solar Radiation (Negative Value Correction)', fontweight='bold')
        ax3.set_ylabel('Solar Radiation (W/m²)')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        ax3.axhline(0, color='red', linestyle='--', alpha=0.5, label='Zero line')
        
        # Highlight negative values
        negative_mask = original_data['global_radiation'] < 0
        if negative_mask.any():
            ax3.scatter(original_data.loc[negative_mask, 'timestamp'],
                       original_data.loc[negative_mask, 'global_radiation'],
                       color='red', s=30, marker='v', label='Negative values')
        
        # Plot 4: Data completeness comparison
        ax4 = axes[1, 1]
        
        # Calculate completeness for key variables
        original_completeness = {
            'Temperature': (1 - original_data['air_temperature'].isna().mean()) * 100,
            'Humidity': (1 - original_data['relative_humidity'].isna().mean()) * 100,
            'Wind': (1 - original_data['wind_speed'].isna().mean()) * 100,
            'Radiation': (1 - original_data['global_radiation'].isna().mean()) * 100
        }
        
        if len(processed_data.columns) > 4:  # Has processed data
            var_mapping = {'air_temperature': 'Tair', 'relative_humidity': 'RH', 
                          'wind_speed': 'U', 'global_radiation': 'Kdown'}
            processed_completeness = {}
            for orig_var, proc_var in var_mapping.items():
                if proc_var in processed_data.columns:
                    processed_completeness[orig_var.replace('_', ' ').title()] = (
                        1 - processed_data[proc_var].isna().mean()) * 100
        else:
            processed_completeness = original_completeness.copy()
        
        variables = list(original_completeness.keys())
        orig_values = list(original_completeness.values())
        proc_values = [processed_completeness.get(var, orig_values[i]) 
                      for i, var in enumerate(variables)]
        
        x = np.arange(len(variables))
        width = 0.35
        
        bars1 = ax4.bar(x - width/2, orig_values, width, label='Original', alpha=0.7, color='lightblue')
        bars2 = ax4.bar(x + width/2, proc_values, width, label='Processed', alpha=0.7, color='lightgreen')
        
        ax4.set_title('Data Completeness Comparison', fontweight='bold')
        ax4.set_ylabel('Completeness (%)')
        ax4.set_xticks(x)
        ax4.set_xticklabels(variables, rotation=45, ha='right')
        ax4.legend()
        ax4.grid(True, alpha=0.3, axis='y')
        ax4.set_ylim(0, 105)
        
        # Add value labels on bars
        for bar, value in zip(bars1, orig_values):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f'{value:.1f}%', ha='center', va='bottom', fontsize=8)
        
        for bar, value in zip(bars2, proc_values):
            ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                    f'{value:.1f}%', ha='center', va='bottom', fontsize=8)
        
        plt.tight_layout()
        plt.show()
        
        # Print summary statistics
        print("\n📊 Data Quality Summary:")
        print("=" * 40)
        
        print(f"📈 Original data points: {len(original_data):,}")
        print(f"📈 Missing humidity values: {original_data['relative_humidity'].isna().sum()}")
        print(f"🌡️ Temperature outliers (>35°C): {(original_data['air_temperature'] > 35).sum()}")
        print(f"☀️ Negative radiation values: {(original_data['global_radiation'] < 0).sum()}")
        
        if len(processed_data.columns) > 4:
            print(f"\n✅ After processing:")
            print(f"📈 Processed data points: {len(processed_data):,}")
            if 'RH' in processed_data.columns:
                print(f"📈 Missing humidity values: {processed_data['RH'].isna().sum()}")
            if 'Kdown' in processed_data.columns:
                print(f"☀️ Negative radiation values: {(processed_data['Kdown'] < 0).sum()}")
        
        return original_data, processed_data
        
    except Exception as e:
        print(f"⚠️ Comparison failed: {e}")
        return None, None

# Compare before and after
original_data, processed_data = compare_before_after()

## Step 8: Create Final SUEWS-Ready Dataset

Let's demonstrate how to create a final dataset that's ready for SUEWS simulation.

In [ ]:
def create_suews_ready_dataset():
    """Create a final SUEWS-ready dataset from processed data."""
    
    print("🎯 Creating final SUEWS-ready dataset...")
    
    # Check if we have processed data, otherwise use original
    data_files = [
        'sample_data/weather_station_fixed.txt',
        'sample_data/weather_station_assessed.txt'
    ]
    
    processed_df = None
    for file_path in data_files:
        try:
            processed_df = pd.read_csv(file_path, delim_whitespace=True)
            print(f"✅ Using processed data from: {file_path}")
            break
        except FileNotFoundError:
            continue
    
    if processed_df is None:
        print("⚠️ No processed data found, creating from original data...")
        original_df = pd.read_csv('sample_data/weather_station.csv')
        
        # Create SUEWS format manually
        processed_df = pd.DataFrame({
            'Year': 2023,
            'DOY': pd.to_datetime(original_df['timestamp']).dt.dayofyear,
            'Hour': pd.to_datetime(original_df['timestamp']).dt.hour,
            'Min': 0,
            'Tair': original_df['air_temperature'].fillna(method='linear'),
            'RH': original_df['relative_humidity'].fillna(method='linear'),
            'U': original_df['wind_speed'],
            'WDir': original_df['wind_direction'],
            'Pres': original_df['pressure'],
            'Kdown': np.maximum(0, original_df['global_radiation']),  # Fix negative values
            'Rain': original_df['precipitation']
        })
    
    # Ensure all required SUEWS variables are present
    required_vars = ['Year', 'DOY', 'Hour', 'Min', 'Tair', 'RH', 'U', 'Pres', 'Kdown', 'Rain']
    missing_vars = [var for var in required_vars if var not in processed_df.columns]
    
    if missing_vars:
        print(f"⚠️ Missing required variables: {missing_vars}")
        print("Available variables:", list(processed_df.columns))
        
        # Add missing time variables if needed
        if 'Year' not in processed_df.columns:
            processed_df['Year'] = 2023
        if 'DOY' not in processed_df.columns:
            processed_df['DOY'] = 182 + np.arange(len(processed_df)) // 24  # Start from July 1st
        if 'Hour' not in processed_df.columns:
            processed_df['Hour'] = np.arange(len(processed_df)) % 24
        if 'Min' not in processed_df.columns:
            processed_df['Min'] = 0
    
    # Reorder columns to match SUEWS format
    suews_columns = ['Year', 'DOY', 'Hour', 'Min', 'Tair', 'RH', 'U', 'WDir', 'Pres', 'Kdown', 'Rain']
    available_columns = [col for col in suews_columns if col in processed_df.columns]
    
    final_df = processed_df[available_columns].copy()
    
    # Data quality checks
    print("\n🔍 Final dataset quality checks:")
    
    for var in ['Tair', 'RH', 'U', 'Kdown']:
        if var in final_df.columns:
            missing_pct = (final_df[var].isna().mean()) * 100
            if missing_pct < 5:
                status = "✅"
            elif missing_pct < 20:
                status = "⚠️"
            else:
                status = "❌"
            print(f"   {status} {var}: {missing_pct:.1f}% missing")
    
    # Check value ranges
    range_checks = {
        'Tair': (-30, 50),
        'RH': (0, 100),
        'U': (0, 30),
        'Kdown': (0, 1200)
    }
    
    print("\n🌡️ Value range checks:")
    for var, (min_val, max_val) in range_checks.items():
        if var in final_df.columns:
            out_of_range = ((final_df[var] < min_val) | (final_df[var] > max_val)).sum()
            if out_of_range == 0:
                status = "✅"
            elif out_of_range < len(final_df) * 0.01:  # <1%
                status = "⚠️"
            else:
                status = "❌"
            print(f"   {status} {var}: {out_of_range} values outside range [{min_val}, {max_val}]")
    
    # Save final dataset
    output_file = 'sample_data/final_suews_forcing.txt'
    final_df.to_csv(output_file, sep=' ', index=False, float_format='%.3f')
    
    print(f"\n💾 Final SUEWS-ready dataset saved: {output_file}")
    print(f"📊 Dataset shape: {final_df.shape}")
    print(f"📅 Time period: {final_df.shape[0]} hours")
    print(f"🗂️ Variables: {', '.join(final_df.columns)}")
    
    # Show sample of final data
    print("\n📋 Sample of final dataset:")
    print(final_df.head(10).round(3))
    
    return final_df

# Create final dataset
final_dataset = create_suews_ready_dataset()

## Step 9: Validate Final Dataset

Let's validate our final dataset against SUEWS requirements using the MCP server.

In [ ]:
async def validate_final_dataset():
    """Validate the final SUEWS-ready dataset."""
    if not connection_ok:
        print("❌ Skipping - no MCP connection")
        return
    
    async with create_client("suews-mcp") as client:
        print("🔍 Final validation of SUEWS-ready dataset...")
        
        try:
            validation = await client.call_tool("preprocess_forcing", {
                "input_file": "sample_data/final_suews_forcing.txt",
                "validate_energy_balance": False,  # No energy balance in weather station data
                "auto_fix_issues": False,  # Just validate
                "target_timestep": 3600
            })
            
            print("✅ Final Dataset Validation:")
            print("=" * 35)
            print(validation.content[0].text)
            
            # Check if validation indicates readiness for SUEWS
            validation_text = validation.content[0].text.upper()
            if "SUCCESS" in validation_text and "READY" in validation_text:
                print("\n🎉 Dataset is ready for SUEWS simulation!")
            elif "SUCCESS" in validation_text:
                print("\n✅ Dataset validation passed. Minor issues may exist but simulation should work.")
            else:
                print("\n⚠️ Dataset has issues that should be addressed before simulation.")
            
            return validation
            
        except Exception as e:
            print(f"⚠️ Final validation failed: {e}")
            return None

# Validate final dataset
if connection_ok and final_dataset is not None:
    final_validation = await validate_final_dataset()
elif final_dataset is not None:
    print("📊 Manual validation of final dataset:")
    print(f"   ✅ Dataset shape: {final_dataset.shape}")
    print(f"   ✅ Required time variables present: Year, DOY, Hour, Min")
    print(f"   ✅ Required meteorological variables: {', '.join(['Tair', 'RH', 'U', 'Kdown', 'Rain'])}")
    print(f"   ✅ Dataset appears ready for SUEWS simulation")

## Summary and Best Practices

Congratulations! You've completed a comprehensive data preprocessing workflow for SUEWS. Here's what you accomplished:

### ✅ What you did:
1. **Created sample data** with realistic meteorological variables and common issues
2. **Assessed data quality** using MCP preprocessing tools
3. **Converted data formats** from Excel to SUEWS format with column mapping
4. **Validated energy balance** for flux tower data
5. **Applied automatic fixes** to common data issues
6. **Created final SUEWS-ready** forcing file
7. **Visualized improvements** in data quality

### 📊 Key Data Quality Issues Addressed:
- **Missing data**: Gap-filled humidity measurements
- **Outliers**: Removed unrealistic temperature spikes  
- **Negative values**: Fixed negative solar radiation measurements
- **Energy balance**: Validated flux tower energy closure
- **Format conversion**: Standardized column names and units
- **Temporal consistency**: Ensured regular time steps

### 🚀 Best Practices for Real Data:

#### 1. Data Source Hierarchy
```python
# Priority order for merging multiple datasets:
# 1. High-quality flux tower measurements
# 2. Standard weather station data  
# 3. Reanalysis/gridded data for gap filling
```

#### 2. Quality Control Thresholds
- **Temperature**: -50°C to +60°C (climate dependent)
- **Humidity**: 0% to 100%
- **Wind speed**: 0 to 50 m/s (>30 m/s rare)
- **Solar radiation**: 0 to 1400 W/m²
- **Energy balance closure**: 80-120% acceptable

#### 3. Missing Data Handling
- **<5% missing**: Linear interpolation acceptable
- **5-20% missing**: Use climatological patterns
- **>20% missing**: Find alternative data sources

#### 4. Energy Balance Requirements
For flux tower data: **QN = QH + QE + QS ± residual**
- Residual should be <20% of QN
- Bowen ratio (QH/QE) should be 0.1-5.0 for most urban areas

### 🔧 Common Data Issues and Solutions:

| Issue | Symptoms | Solution |
|-------|----------|----------|
| **Clock errors** | Time jumps, irregular intervals | Use `target_timestep` parameter |
| **Unit mismatches** | Unrealistic values | Check units in column mapping |
| **Instrument drift** | Gradual bias over time | Use reference data for correction |
| **Sensor failures** | Long periods of constant values | Flag and interpolate |
| **Format issues** | Parsing errors | Use appropriate `input_format` |

### 📈 Next Steps:

1. **Use Your Own Data**:
   ```python
   # Process your meteorological data
   await client.call_tool("preprocess_forcing", {
       "input_file": "your_data.csv",
       "auto_fix_issues": True,
       "validate_energy_balance": True
   })
   ```

2. **Run SUEWS Simulation**:
   ```python
   # Use your processed data in simulation
   await client.call_tool("run_simulation", {
       "forcing_path": "sample_data/final_suews_forcing.txt",
       "config_path": "residential_config.yml"
   })
   ```

3. **Advanced Preprocessing**:
   - Multi-source data merging
   - Quality control with observations
   - Long-term trend analysis
   - Climate change scenario preparation

### 📚 Additional Resources:
- [SUEWS MCP API Reference](../docs/api_reference.md)
- [Data Preprocessing Workflow Guide](../docs/examples/data_preprocessing_workflow.md)
- [Urban Heat Island Example](../docs/examples/urban_heat_island_study.md)
- [FAQ & Troubleshooting](../docs/faq.md)

### 🆘 Need Help?
- Check the [FAQ & Troubleshooting Guide](../docs/faq.md)
- Visit [SUEWS Documentation](https://suews.readthedocs.io/)
- Open an issue on [GitHub](https://github.com/UMEP-dev/SUEWS/issues)

Your data is now ready for urban climate modeling! 📊🏙️

In [ ]:
# Final summary
print("🎉 DATA PREPROCESSING TUTORIAL COMPLETE!")
print("=" * 50)

if connection_ok:
    print("✅ MCP server connection: SUCCESS")
else:
    print("❌ MCP server connection: FAILED")

if final_dataset is not None:
    print("✅ Sample data creation: SUCCESS")
    print("✅ Data preprocessing: COMPLETED")
    print(f"📊 Final dataset: {final_dataset.shape[0]:,} data points")
    print(f"🗂️ Variables: {len(final_dataset.columns)}")
    print("💾 Output: sample_data/final_suews_forcing.txt")
else:
    print("⚠️ Data preprocessing: INCOMPLETE")

print("\n🚀 Ready to run SUEWS simulations with your processed data!")
print("\n📖 Next: Try the Basic Simulation notebook with your processed forcing data")